In [1]:
! pip install python-dotenv langchain langchain-community langchain-openai langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.6.0b3

In [2]:
"""
The environment variables are loaded from the `.env` file in the same directory as this notebook.
"""
import os
from dotenv import load_dotenv

_=load_dotenv()

os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

In [3]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

In [4]:
#check if the file exists
if not os.path.exists("./data/hyde.pdf"):
    raise Exception("The file ./data/hyde.pdf does not exist.")

pdf_path="./data/hyde.pdf"

In [7]:
print(doc_intelligence_endpoint)
print(doc_intelligence_key)

https://docint-westus2.cognitiveservices.azure.com/
be98d969c2e04ad69f206d56d1bbacab


In [9]:
# Initiate Azure AI Document Intelligence to load the document. You can either specify file_path or url_path to load the document.
loader = AzureAIDocumentIntelligenceLoader(file_path=pdf_path, api_key = doc_intelligence_key, api_endpoint = doc_intelligence_endpoint, api_model="prebuilt-layout", api_version="2024-07-31-preview")
docs = loader.load()

# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    #('####', 'Header 4'),
    #('#####'    , 'Header 5'),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

docs_string = docs[0].page_content
splits = text_splitter.split_text(docs_string)

print("Length of splits: " + str(len(splits)))

Length of splits: 21


In [8]:
splits

[Document(page_content='Precise Zero-Shot Dense Retrieval without Relevance Labels\n===  \nLuyu Gao \\*\\* Xueguang Ma \\*\\* Jimmy Lin Jamie Callant  \nLanguage Technologies Institute, Carnegie Mellon University  \nDavid R. Cheriton School of Computer Science, University of Waterloo {luyug, callan} @cs.cmu.edu, {x93ma, jimmylin} @uwaterloo.ca'),
 Document(metadata={'Header 1': 'Abstract'}, page_content="arXiv:2212.10496v1 [cs.IR] 20 Dec 2022  \nWhile dense retrieval has been shown effec- tive and efficient across tasks and languages, it remains difficult to create effective fully zero-shot dense retrieval systems when no rel- evance label is available. In this paper, we recognize the difficulty of zero-shot learning and encoding relevance. Instead, we pro- pose to pivot through Hypothetical Document Embeddings (HyDE). Given a query, HyDE first zero-shot instructs an instruction-following language model (e.g. InstructGPT) to gen- erate a hypothetical document. The docu- ment captures r

## Undersatd the Azure Docuement Intelligence chunking
### TODO - Provide explanation here and why it's good for RAG 